In [ ]:
import json
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

connection_parameters = json.load(open('../connection.json'))
session = Session.builder.configs(connection_parameters).create()

### 1. Build docker image and push the image to image registry

- Run the follolwing commands from terminal. <b>Set your working directory to 3. Text2SQL. </b>

``` bash
docker build --no-cache --platform linux/amd64 -t ORGNAME-ACCTNAME.registry.snowflakecomputing.com/llmdemo/public/images/audiollm:latest . 

docker push ORGNAME-ACCTNAME.registry.snowflakecomputing.com/llmdemo/public/images/audiollm:latest
```

###  2. Creating Text2SQL SPC Service

Update the image line in [llm-text2sql.yaml](./llm-text2sql.yaml) to provide your orgname-accountname values

ex. image: <i>myorg-myaccount</i>.registry.snowflakecomputing.com/pr_llmdemo/public/image_repo/whisper-audio2text:latest

PS: <b>Run following commands using the SPCS Role, not accountadmin! (this should already be set in your [connection.json](../connection.json) file) </b>


In [ ]:

session.file.put("./llm-text2sql.yaml", "@specs",auto_compress=False)

In [ ]:
# Create the required stage
stages=['LLM_WORKSPACE']
for stg in stages:
    session.sql(f'''
                CREATE STAGE IF NOT EXISTS {stg} ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
                DIRECTORY = (ENABLE = TRUE);
                ''').collect()

In [ ]:
session.sql('''
create service if not exists llama_text2sql_svc
in compute pool PR_GPU_7
from @specs
SPECIFICATION_FILE='llm-text2sql.yaml'
EXTERNAL_ACCESS_INTEGRATIONS = (ALLOW_ALL_EAI)
            ''').collect()

> The service must be in Ready State to proceed. Run the following command to confirm before proceeding to next step.

In [ ]:
# Service activation will take several minutes, so just rerun this command until you see status READY (patience is a virtue)
import ast
res=session.sql(''' 
SELECT SYSTEM$GET_SERVICE_STATUS('llama_text2sql_svc',1)
''').collect()[0][0]
ast.literal_eval(res)[0]

>  Be sure the service is in the Ready State before procedding.

### 4. Access Jupyter Lab Endpoint running in SPCS

Run the following query to get the api endpoint (ingress_url) for the <b> jupyter lab </b>

In [ ]:
session.sql('''show endpoints in service llama_text2sql_svc;
            ''').collect()[1]

# Row(name='llm-audio-ep', port=8888, port_range=None, protocol='HTTP', is_public='true', ingress_url='yjb2m-sfsenorthamerica-demo-psheehan.snowflakecomputing.app')

### 5. Perform the following actions in the Jupyter lab

<b> 1. Open the terminal in JupyterLab and run the following shell script. <i>Wait until download completes before proceeding to next step.</i>  </b>

- sh download_model.sh

This file will download the Numberstation model from hugging face to a stage

<b> 2. Run the FineTuneModel.ipynb</b>

In this step we train (fine tune) a NStext2SQL model

<b> 3. Open the terminal in JupyterLab and run the following shell script.</b> This will expose the fine-tuned model as an api endpoint.

sh RunFastAPI.sh

### 6. Execute the following commands to create the service function to call our fine-tuned model

In [ ]:
session.sql('''CREATE OR REPLACE FUNCTION text2sql(text TEXT)
RETURNS VARIANT
SERVICE=llama_text2sql_svc
ENDPOINT=api
AS '/text2sql' ''').collect()


In [ ]:
#Testing the function
session.sql('''
select text2sql('What is the distinct purpose of the calls in the last month?')::string;
''').collect()